## Install and Import Package

In [ ]:
!pip install ultralytics

In [ ]:
import zipfile
import requests
import cv2
import matplotlib.pyplot as plt
import glob 
import random
import os

## Clone Git Repo

In [ ]:
!git clone https://github.com/ThatCoderMan/Hay_test_task.git
!cp -rf ./Hay_test_task/* ./
!rm -r Hay_test_task/

## Download the Dataset

In [ ]:
os.makedirs('datasets', exist_ok=True)

In [ ]:
# !pip install roboflow
# 
# from roboflow import Roboflow
# rf = Roboflow(api_key="VDs1nenfrsfHSo45TXZj")
# project = rf.workspace("hay").project("hay-yypkw")
# dataset = project.version(1).download("yolov8")

In [ ]:
%cd datasets

In [ ]:
!curl -L "https://app.roboflow.com/ds/Z4MWCxM9E7?key=fk3g2mGGeB" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%cd ..

In [ ]:
%pwd

## Visualize Images from the Dataset

In [ ]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [ ]:
def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # Denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        thickness = max(2, int(w/275))
                
        cv2.rectangle(
            image, 
            (xmin, ymin), (xmax, ymax),
            color=(0, 0, 255),
            thickness=thickness
        )
    return image

In [ ]:
# Function to plot images with the bounding boxes.
def plot(image_paths, label_paths, num_samples):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.JPG'))
    
    all_images.sort()

    num_images = len(all_images)
    
    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image_name = all_images[j]
        image_name = '.'.join(image_name.split(os.path.sep)[-1].split('.')[:-1])
        image = cv2.imread(all_images[j])
        with open(os.path.join(label_paths, image_name+'.txt'), 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line[2:]
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=1)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize a few training images.
plot(
    image_paths='datasets/train/images/', 
    label_paths='datasets/train/labels/',
    num_samples=4,
)

## Dataset YAML File

In [ ]:
%%writefile hay.yaml
path: ''
train: 'train/images'
val: 'valid/images'

# class names
names: 
  0: 'hay'

## YOLOv8 Nano Training

In [ ]:
# Sample training for 5 epoch.
EPOCHS = 10
!yolo task=detect mode=train model=yolov8n.pt epochs={EPOCHS} batch=8 name=hay_model data=hay.yaml

## Evaluation on Validation Images

In [ ]:
!yolo task=detect mode=val model=runs/detect/hay_model5/weights/best.pt name=hay_model_eval data=hay.yaml

In [ ]:
!yolo export model=runs/detect/hay_model/weights/best.pt format=onnx

## Inference on Validation Images

In [ ]:
!yolo task=detect \
mode=predict \
model=runs/detect/hay_model5/weights/best.pt \
source=data/videos/original_video \
name=result \
hide_labels=True

In [ ]:
!ffmpeg -f image2 -r 24 -i ./runs/detect/results/frame_%04d.jpg -vcodec libx264 -crf 18  -pix_fmt yuv420p result.mp4 

## Visualize Validation Results

In [ ]:
# Plot and visualize images in a 2x2 grid.
def visualize(result_dir, num_samples=4):
    """
    Function accepts a list of images and plots
    them in a 2x2 grid.
    """
    plt.figure(figsize=(20, 12))
    image_names = glob.glob(os.path.join(result_dir, '*.jpg'))
    random.shuffle(image_names)
    for i, image_name in enumerate(image_names):
        image = plt.imread(image_name)
        plt.subplot(2, 2, i+1)
        plt.imshow(image)
        plt.axis('off')
        if i == num_samples-1:
            break
    plt.tight_layout()
    plt.show()

In [ ]:
visualize('runs/detect/result/')